# 输出解析器
* 格式化LLM的答案。

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

* 注意：由于目前市场上最佳的 LLM 是 OpenAI，我们将默认使用它。您将在下一课中看到如何连接其他开源 LLM，例如 Llama3 或 Mistral。

## LLM 模型
* chatGPT-4 发布前的趋势。
* 在 [这里](https://python.langchain.com/v0.1/docs/modules/model_io/llms/) 查看有关 LLM 模型的 LangChain 文档。

In [5]:
from langchain_openai import OpenAI

llmModel = OpenAI()

## 聊天模型
* chatGPT-4 发布后的总体趋势。
    * 通常称为“聊天机器人”。
    * 人类与 AI 之间的对话。
    * 可以有一个系统提示来定义 AI 的语气或角色。
* 请查看 LangChain 文档中有关聊天模型的内容 [这里](https://python.langchain.com/v0.1/docs/modules/model_io/chat/)。
* 默认情况下，我们将使用 ChatOpenAI。请查看有关它的 LangChain 文档页面 [这里](https://python.langchain.com/v0.1/docs/integrations/chat/openai/)。

In [6]:
from langchain_openai import ChatOpenAI

chatModel = ChatOpenAI(model="gpt-3.5-turbo-0125")

## 解析输出
* 请查看相应的 LangChain 文档页面 [这里](https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/)。
* 语言模型输出文本。但有时我们希望以不同的格式获得这些答案，例如 JSON 字典或 XML 文档。为了实现这一点，我们使用输出解析器。

In [7]:
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers.json import SimpleJsonOutputParser

json_prompt = PromptTemplate.from_template(
    "Return a JSON object with an `answer` key that answers the following question: {question}"
)

json_parser = SimpleJsonOutputParser()

json_chain = json_prompt | llmModel | json_parser

#### 先前的提示模板包含了解析器说明

In [8]:
json_parser.get_format_instructions()

'Return a JSON object.'

In [9]:
json_chain.invoke({"question": "What is the biggest country?"})

{'answer': 'Russia'}

#### Optionally, you can use Pydantic to define a custom output format

In [10]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

In [11]:
# Define a Pydantic Object with the desired output format.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

In [12]:
# Define the parser referring the Pydantic Object
parser = JsonOutputParser(pydantic_object=Joke)

# Add the parser format instructions in the prompt definition.
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Create a chain with the prompt and the parser
chain = prompt | chatModel | parser

chain.invoke({"query": "Tell me a joke."})

{'setup': "Why couldn't the bicycle find its way home?",
 'punchline': 'Because it lost its bearings!'}